## TOC:
* [Cubic Spline of Trajectory points](#cubic)
* [Why Quintic Polynomials](#quintic)
* [Lattice Planner with PID controller](#lattice)

## Cubic Spline of Trajectory points <a class="anchor" id="lattice"></a>
三次样条插值在每个相邻点间使用独立的三次函数拟合曲线，插值函数可以表达为
$$
S_j(x)=a_j+b_j(x-x_j)+c_j(x-x_j)^2+d_j(x-x_j)^3, \quad x_j < x < x_{j+1}
$$
$x_j$ 是序列的第$j$个点, $a_j$, $b_j$, $c_j$, $d_j$ 是曲线的系数。上述曲线的未知参数有$4$个，若点序列长度为$N$，那么三次样条插值的参数量为$4N$。三次样条插值满足以下的条件：
- $S_j(x_j)=y_j$ （曲线必须经过给定样本点）
- $S_j(x_{j+1})=S_{j+1}(x_{j+1})=y_{j+1}$ （相邻曲线首尾相接）
- $S'_j(x_{j+1})=S'_{j+1}(x_{j+1})$ （切线连续）
- $S''_j(x_{j+1})=S''_{j+1}(x_{j+1})$ （曲率连续）
- $S''_0(0)=S''_{n+1}(x_{n})=0$ （边界条件）

### Calculate the coeffcients
#### calculate $a_j$

系数 $a_j$ 可以直接由点的$y$值给定:
$$
a_j = y_i
$$.

#### calculate $c_j$

系数 $c_j$ 可以通过求解如下的线性方程组$Ac_j = B$得到。

矩阵 $A$ 和 $B$ 的定义如下:

$$
	A=\left[\begin{array}{cccccc}
	1 & 0 & 0 & 0 & \cdots & 0 \\
	h_{0} & 2\left(h_{0}+h_{1}\right) & h_{1} & 0 & \cdots & 0 \\
	0 & h_{1} & 2\left(h_{1}+h_{2}\right) & h_{2} & \cdots & 0 \\
	0 & 0 & h_{2} & 2\left(h_{2}+h_{3}\right) & \cdots & 0 \\
	0 & 0 & 0 & h_{3} & \ddots & \\
	\vdots & \vdots & & & & \\
	0 & 0 & 0 & \cdots & 0 & 1
	\end{array}\right]
$$
$$
	B=\left[\begin{array}{c}
	0 \\
	\frac{3}{h_{1}}\left(a_{2}-a_{1}\right)-\frac{3}{h_{0}}\left(a_{1}-a_{0}\right) \\
	\vdots \\
	\frac{3}{h_{n-1}}\left(a_{n}-a_{n-1}\right)-\frac{3}{h_{n-2}}\left(a_{n-1}-a_{n-2}\right) \\
	0
	\end{array}\right]
$$
$h_{i} = x_{i+1} - x_{i}$ 是第$i$个点和第$i+1$个点$x$值的差.

Hint:
$$
h_{i} = x_{i+1} - x_{i} \tag{1}
$$
$$
a_j + b_jh_j + c_jh_j^2+d_jh_j^3 = a_{j+1} \tag{2}
$$
$$
b_j + 2c_jh_j + 3d_jh_j^2 = b_{j+1} \tag{3}
$$
$$
2c_j + 6d_jh_j = 2c_{j+1} \tag{4}
$$
$$
\frac{(4) * h_j}{2} - (3) \Rightarrow b_j - b_{j+1}  = -c_jh_j - c_{j+1}h_j \tag{5}
$$
$$
\frac{(2) * 3}{h_j} - (5) \Rightarrow 3\frac{a_j}{h_j} + 3b_j + 2c_j*h_j = 3\frac{a_{j+1}}{h_j}-c_{j+1}h_j \\
b_j = \frac{a_{j+1}}{h_j} - \frac{a_j}{h_j} - \frac{2}{3}c_jh_j - \frac{1}{3}c_{j+1}h_j \\
b_{j+1} = \frac{a_{j+2}}{h_{j+1}} - \frac{a_{j+1}}{h_{j+1}} - \frac{2}{3}c_{j+1}h_{j+1} - \frac{1}{3}c_{j+2}h_{j+1} \\ 
b_j - b_{j+1} = \frac{1}{h_j}(a_{j+1} - a_j) - \frac{1}{h_{j+1}}(a_{j+2}-a_{j+1})-\frac{2}{3}(c_jh_j-c_{j+1}h_{j+1}) \\ - \frac{1}{3}(c_{j+1}h_j - c_{j+2}h_{j+1})
\tag{6}
$$
$$
(5), (6) \Rightarrow h_jc_j+2(h_j+h_{j+1})c_{j+1} + h_{j+1}c_{j+2} = \frac{3}{h_{j+1}}(a_{j+2} - a_{j+1}) - \frac{3}{h_j}(a_{j+1} - a_j) \tag{7}
$$

#### calculate $d_j$
系数 $d_j$ 由下列公式得到:
$$
	d_{j}=\frac{c_{j+1}-c_{j}}{3 h_{j}}
$$

#### calculate $b_j$
系数$b_j$由下列公式得到:
$$
	b_{i}=\frac{1}{h_i}(a_{i+1}-a_{i})-\frac{h_i}{3}(2c_{i}+c_{i+1})
$$


## Why Quintic Polynomials? <a class="anchor" id="quintic"></a>
在车辆行驶中，舒适度是非常重要的一个指标，在物理中舒适度的物理量是跃度，即Jerk，定义是
$$
Jerk = \frac{da}{dt}
$$
其中$a$表示加速度，即$Jerk$是加速度的导数，$Jerk$的绝对值越小意味着的变化越平缓，$a$的变化越平缓意味着越舒适

在路径规划中，我们用$s$相对于时间$t$的函数$s=f(t)$来表示轨迹，那么$Jerk = \frac{d^3f}{dt^3}$，为了使在$[0,T]$内$Jerk$的变化的比较平缓，我们可以构造以下的优化问题：
$$
\mathrm{find}\, f(t) \, s.t. \, min\int_0^T(\frac{d^3f}{dt^3})^2dt
$$
并且我们的边界优化条件要求
$$
s(0) = s_0 \qquad s(T) = s_n \\
s'(0) = v_0 \qquad s'(T) = v_n \\
s''(0) = a_0 \qquad s''(T) = a_n
$$

对初始状态和终止状态进行差分，记
$$
S(T) - S(0) = s_n - s_0 = \int_0^Tf'dt = C_0 \\
S'(T) - S'(0) = v_n - v_0 = \int_0^Tf''dt = C_1 \\
S''(T) - S''(0) = a_n - a_0 = \int_0^Tf'''dt = C_2
$$
那么原问题就是求解满足约束$\int_0^T(f'-\frac{C_0}{T})dt = 0$, $\int_0^T(f''-\frac{C_1}{T})dt = 0$, $\int_0^T(f'''-\frac{C_2}{T})dt = 0$时，$\int_0^T(\frac{d^3f}{dt^3})^2dt$取最小值的函数$f(t)$

那么根据Lagrange乘子法，有
$$
\int_0^T(\frac{d^3f}{dt^3})^2dt + \lambda_1\int_0^T(f'-\frac{C_0}{T})dt + \lambda_2\int_0^T(f''-\frac{C_1}{T})dt + \lambda_3\int_0^T(f'''-\frac{C_2}{T}) \\
= \int_0^T(\lambda_1f'+\lambda_2f''+\lambda_3f'''+f'''^2-\lambda1\frac{C_0}{T}-\lambda2\frac{C_1}{T}-\lambda3\frac{C_2}{T})dt \\
= \int_0^TLdt
$$

那么根据广义Euler-Lagrange方程有
$$
\frac{\partial L}{\partial f} - \frac{d}{dt}\frac{\partial L}{\partial f'} + \frac{d^2}{dt^2}\frac{\partial L}{\partial f''} - \frac{d^3}{dt^3}\frac{\partial L}{\partial f'''} = 0
$$
又有
$$
\frac{\partial L}{\partial f}=0,\,\frac{d}{dt}\frac{\partial L}{\partial f'}=\lambda_1,\,\frac{d^2}{dt^2}\frac{\partial L}{\partial f''}=\lambda_2,\,\frac{d^3}{dt^3}\frac{\partial L}{\partial f'''}=\lambda_3+2f'''
$$
可以得到$f^{(6)}(t)=0$，所以$f$为五次多项式时是泛函$\int_0^T(\frac{d^3f}{dt^3})^2dt$取极值的解函数

### Hint: Euler-Lagrange Equation
欧拉—拉格朗日方程是泛函取极值的条件，对于一个泛函$\mathcal{F}(f) \in (\mathbb{R} \mapsto \mathbb{R}) \mapsto \mathbb{R}$，考虑常见的积分操作，对于给定的泛函$\mathcal{F}=\int_{x_1}^{x_2}L(x,f(x),f'(x))dx$，考虑取到极值时的解$\hat{f}$，我们可以用一个放缩系数$\epsilon$和一个扰动函数$\eta \in \mathbb{R} \mapsto \mathbb{R}$来表示函数集合中的其他函数$f=\hat{f}+\epsilon\eta$，那么我们可以将泛函$\mathcal{F}$写成:
$$
\mathcal{F}(f) = \int_{x_1}^{x_2}L(x,f(x),f'(x))dx = \int_{x_1}^{x_2}L(x, \hat{f}(x)+\epsilon\eta(x), \hat{f}'(x)+\epsilon\eta'(x))dx
$$
上面的泛函中的扰动可以通过系数$\epsilon$控制，那么应用费马引理我们有
$$
\frac{\partial F(f)}{\partial \epsilon} = \int_{x_1}^{x_2}(\frac{\partial L}{\partial f}\eta + \frac{\partial L}{\partial f'}\eta')dx = 0
$$
拆开积分
$$
\frac{\partial F(f)}{\partial \epsilon} = \int_{x_1}^{x_2}{\frac{\partial L}{\partial f}\eta}dx + \int_{x_1}^{x_2}{\frac{\partial L}{\partial f'}\eta'}dx
$$
将右边分部积分
$$
\int_{x_1}^{x_2}{\frac{\partial L}{\partial f'}\eta'}dx = \int_{x_1}^{x_2}\frac{\partial L}{\partial f'} d\eta = \left.\eta\frac{\partial L}{\partial f'}\right\vert_{x_1}^{x_2} - \int_{x_1}^{x_2}\eta d(\frac{\partial L}{\partial f'})
$$
又因为对于边界$x_1$和$x_2$来说，扰动函数$\eta$的值应该为$0$，所以有
$$
\int_{x_1}^{x_2}(\frac{\partial L}{\partial f'}\eta')dx = -\int_{x_1}^{x_2}\eta d(\frac{\partial L}{\partial f'}) = -\int_{x_1}^{x_2} \eta\frac{d}{dx}(\frac{\partial L}{\partial f'})dx
$$
结合上述式子，有
$$
\frac{\partial F(f)}{\partial \epsilon} = \int_{x_1}^{x_2}{\frac{\partial L}{\partial f}\eta}dx -\int_{x_1}^{x_2} \eta\frac{d}{dx}(\frac{\partial L}{\partial f'})dx \\
= \int_{x_1}^{x_2}\eta(\frac{\partial L}{\partial f} - \frac{d}{dx}(\frac{\partial L}{\partial f'}))dx
$$
为了使上述的式子对任意的扰动函数$\eta$都生效，可得
$$
\frac{\partial L}{\partial f} - \frac{d}{dx}(\frac{\partial L}{\partial f'}) = 0
$$

## Lattice Planner with PID controller <a class="anchor" id="lattice"></a>
在这小节中，我们将局部规划路径与一个PID控制器相连，控制小车在交通流中进行避障和自动驾驶。

In [1]:
import sys
sys.path.append('../')
import lanelet2
from lanelet2.projection import UtmProjector
import numpy as np
import time
import random
import carla
from ISS.algorithms.planning.global_planner.lanelet2_planner import Lanelet2Planner
from ISS.algorithms.planning.local_planner.lattice_planner import LatticePlanner
from ISS.algorithms.utils.vehicleutils.vehicleutils import CollisionChecker
from ISS.algorithms.control.pid.pid import VehiclePIDController
from ISS.algorithms.localization.gt_carla import GroundTruthLocalizationCarla
from ISS.algorithms.perception.detection_3d.gt import Detection3Dgt, Detection3DgtPred
from ISS.algorithms.utils.dataexchange.multiprocessing.manager import DequeManager
from ISS.algorithms.utils.dataexchange.multiprocessing.proxies import DequeProxy
from multiprocessing import Value

def get_solid_checker(loadedMap):
    ## Get Solid Points...
    inset = set()
    solid_points = []
    solid_id = []
    for lanelet in loadedMap.laneletLayer:
        if 'subtype' not in lanelet.attributes or lanelet.attributes['subtype'] != 'road':
            continue
        left_lane = lanelet.leftBound
        right_lane = lanelet.rightBound        
        if "subtype" in left_lane.attributes and left_lane.attributes['subtype'] == 'solid' and left_lane.id not in inset:
            inset.add(left_lane.id)
            for point in left_lane:
                solid_id.append(left_lane.id)
                solid_points.append((point.x, -point.y))

        if "subtype" in right_lane.attributes and right_lane.attributes['subtype'] == 'solid' and right_lane.id not in inset:
            inset.add(right_lane.id)
            for point in right_lane:
                solid_id.append(right_lane.id)
                solid_points.append((point.x, -point.y))
    ##
    solid_checker = CollisionChecker(solid_points, 4.4, 2.2)
    return solid_checker

def test_pid_mp():    
    lanelet2_town06 = '../resources/maps/Town06_hy.osm'
    projector = UtmProjector(lanelet2.io.Origin(0., 0.))
    loadedMap, load_errors = lanelet2.io.loadRobust(lanelet2_town06, projector) 

    traffic_rules = lanelet2.traffic_rules.create(lanelet2.traffic_rules.Locations.Germany,
                                                  lanelet2.traffic_rules.Participants.Vehicle)
    
    solid_checker = get_solid_checker(loadedMap)
    
    planner = Lanelet2Planner(loadedMap, traffic_rules, solid_checker, reverse_y=True)
    TURNING_RADIUS = 5.

    client = carla.Client('127.0.0.1', 2000)
    world = client.load_world('Town06')
    spawn_points = world.get_map().get_spawn_points()
    start_point = spawn_points[0]
    start_point = (start_point.location.x, start_point.location.y, np.deg2rad(start_point.rotation.yaw))
    end_point = spawn_points[10]
    end_point = (end_point.location.x, end_point.location.y, np.deg2rad(end_point.rotation.yaw))    
    traj = planner.plan(start_point, end_point, TURNING_RADIUS)
    traj.downsample(0.1)  

    lattice_settings = dict()
    lattice_settings['MAX_SPEED'] = 60.0 / 3.6     # maximum speed [m/s]
    lattice_settings['MAX_ACCEL'] = 4.0            # maximum acceleration [m/ss], tesla model3: 6.88
    lattice_settings['MAX_CURVATURE'] = 1.0      # maximum curvature [1/m], tesla model3's turning radius: 5.8    
    lattice_settings['D_S'] = 0.5                  # sample Frenet d
    lattice_settings['D_ROAD_W'] = 1.0             # road width sampling length [m]
    lattice_settings['DT'] = 1.0                   # prediction timestep length (s)
    lattice_settings['dt'] = 0.5                   # sample time
    lattice_settings['MAX_T'] = 6.0                # max prediction time [s]
    lattice_settings['MIN_T'] = 4.0                # min prediction time [s]
    lattice_settings['TARGET_SPEED'] = 15.0 / 3.6  # target speed [m/s]
    lattice_settings['D_T_S'] = 5.0 / 3.6          # target speed sampling length [m/s]
    lattice_settings['N_S_SAMPLE'] = 2             # sampling number of target speed    
    lattice_settings['ROBOT_RADIUS'] = 2.0         # robot radius [m]
    lattice_settings['K_J'] = 0.1
    lattice_settings['K_T'] = 0.1
    lattice_settings['K_D'] = 1.0
    lattice_settings['K_LAT'] = 1.0
    lattice_settings['K_LON'] = 0.8
    
    det3d_settings = dict()
    det3d_settings["T"] = 8     # prediction horizon [s]
    det3d_settings["dt"] = 0.5  # prediction timestep [s]
    det3d_settings["MAX_DISTANCE"] = 50.0  # maximum distance to consider [m]
    
    spawn_points = world.get_map().get_spawn_points()
    spawn_point_ego = spawn_points[0]
    vehicle_bp = world.get_blueprint_library().find('vehicle.tesla.model3')
    vehicle = world.spawn_actor(vehicle_bp, spawn_point_ego)      
    ## NPC Vehicles
    tm = client.get_trafficmanager()
    tm.global_percentage_speed_difference(60)
    npcs = []
    choices = np.random.choice(99, 60, replace=False)
    for index in choices:        
        npc_blueprint = random.choice(world.get_blueprint_library().filter('vehicle.*'))
        npcs.append(world.spawn_actor(npc_blueprint, spawn_points[index+1]))
    for npc_vehicle in npcs:
        npc_vehicle.set_autopilot()
    gt_localization = GroundTruthLocalizationCarla(vehicle)
    lattice =  LatticePlanner(loadedMap, traffic_rules, traj.waypoints, lattice_settings, solid_checker)
    det3d = Detection3DgtPred(det3d_settings)
    controller = VehiclePIDController()
    controller.set_goal(end_point)
    DequeManager.register('DequeProxy', DequeProxy, exposed=['__len__', '__getitem__', 'appendleft', 'append', 'pop', 'popleft'])
    manager = DequeManager()    
    manager.start()
    data_proxies = dict()
    location_queue = manager.DequeProxy(maxlen=1)
    local_traj_queue = manager.DequeProxy(maxlen=1)
    obstacle_detector_queue = manager.DequeProxy(maxlen=1)
    control_queue = manager.DequeProxy(maxlen=1)

    terminating_value = Value('i', 1)
    data_proxies['location_queue'] = location_queue
    data_proxies['local_traj_queue'] = local_traj_queue
    data_proxies['obstacle_detector_queue'] = obstacle_detector_queue
    data_proxies['terminating_value'] = terminating_value
    data_proxies['control_queue'] = control_queue
    process_localization = gt_localization.run_proxies(data_proxies, ip='127.0.0.1', port=2000)
    process_lattice = lattice.run_proxies(data_proxies)
    process_det3d = det3d.run_proxies(data_proxies, ip='127.0.0.1', port=2000, vehicle_id=vehicle.id)
    process_pid = controller.run_proxies(data_proxies)
    processes = [process_localization, process_lattice, process_det3d, process_pid]        
    last_time = time.time()     
    ## Set spectator
    spectator = world.get_spectator()
    ## Dummy Camera
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_transform = carla.Transform(carla.Location(x=-10, z=10), carla.Rotation(pitch=-30))
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
    while terminating_value.value:
        try:
            if vehicle.is_at_traffic_light(): 
                    traffic_light = vehicle.get_traffic_light()
                    if traffic_light.get_state() == carla.TrafficLightState.Red or traffic_light.get_state() == carla.TrafficLightState.Yellow:
                        control = carla.VehicleControl(steer = 0., throttle = 0., brake = 1., hand_brake = True, manual_gear_shift = False)
                        vehicle.apply_control(control)
                        continue
        except:
            pass
        # transform = vehicle.get_transform()
        # spectator.set_transform(carla.Transform(transform.location + carla.Location(z=20), carla.Rotation(yaw=transform.rotation.yaw, pitch=-60)))
        spectator.set_transform(camera.get_transform())
        if time.time() - last_time > 0.5:
            last_time = time.time()
            try:
                tarj = local_traj_queue[-1]
                for point in tarj.waypoints:
                    world.debug.draw_string(carla.Location(point[0], point[1], 0), '*',
                            color=carla.Color(0, 0, 255), life_time=0.5)                                          
                cur_location = location_queue[-1]
                obstacles = obstacle_detector_queue[-1]                
                # collision_point = obstacles.check_point_index((cur_location.x, cur_location.y, cur_location.yaw))                
                # for ind, point in enumerate(obstacles.points):
                #     if ind in collision_point:
                #         world.debug.draw_string(carla.Location(point[0], point[1], 0), '*',
                #             color=carla.Color(255, 0, 0), life_time=0.5)                                          
                #     else:
                #         world.debug.draw_string(carla.Location(point[0], point[1], 0), '*',
                #             color=carla.Color(0, 255, 0), life_time=0.5)                                          
            except Exception as e:
                print(e)
                pass            
        try:
            ## Pass Control Signals to Carla Simulator
            control_ = control_queue[-1]                                                
            control = carla.VehicleControl()
            control.steer = control_.steer
            control.throttle = control_.throttle
            control.brake = control_.brake
            control.hand_brake = control_.hand_brake
            control.manual_gear_shift = control_.manual_gear_shift            
            vehicle.apply_control(control)            
        except:            
            pass
    terminating_value.value = 0
    for process in processes:
        process.join()

test_pid_mp()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
1


Process Process-4:
Traceback (most recent call last):
  File "/home/shaohang/Installation/anaconda3/envs/iss/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/shaohang/Installation/anaconda3/envs/iss/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "ISS/algorithms/perception/detection_3d/gt.pyx", line 153, in ISS.algorithms.perception.detection_3d.gt.Detection3DgtPred.handle
  File "ISS/algorithms/perception/detection_3d/gt.pyx", line 141, in ISS.algorithms.perception.detection_3d.gt.Detection3DgtPred._generate_prediction
  File "/home/shaohang/work_space/autonomous_vehicle/ISS/examples/../ISS/algorithms/utils/dataexchange/planning/trajectory.py", line 58, in get_last_state
    return self.states[-1]
IndexError: list index out of range


deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range
deque index out of range


deque index out of range
deque index out of range
